# Interactive Dashboard using Folium

In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

## Generate map with launch sites

In [4]:
### load data from URL
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

### select the relevant location columns for the map and see their coordinates
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [8]:
### center a new map on NASA Johnson Space Center
nasa_jsc_coord = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_jsc_coord, zoom_start=10)

### add a red circle and popup label
circle = folium.Circle(nasa_jsc_coord, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

### add short marker text
marker = folium.map.Marker(nasa_jsc_coord, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), 
                                                        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',))
site_map.add_child(circle)
site_map.add_child(marker)

In [9]:
### add circles and markers for each launch site on the site 

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for i, row in launch_sites_df.iterrows():
    coord = [row['Lat'],row['Long']]
    label = row['Launch Site']
    ### add a red circle and popup label
    circle = folium.Circle(coord, radius=1000, color='#DDA0DD', fill=True).add_child(folium.Popup(label))

    ### add short marker text
    marker = folium.map.Marker(coord, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), 
                                                            html='<div style="font-size: 12; color:#DDA0DD;"><b>%s</b></div>' % label,))
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

In [11]:
### note that all launch sites are coastal and relatively similar distance from the equator

### add launch outcomes for each site to the site map
# add column with marker color based on class
mcolor = []
for i, row in spacex_df.iterrows():
    if row['class'] == 1 : mcolor.append('#3CB371')
    else : mcolor.append('#DC143C')

spacex_df['marker_color'] = mcolor

marker_cluster = MarkerCluster()

### add outcome markers to marker cluster in the site map
site_map.add_child(marker_cluster)
for i, row in spacex_df.iterrows():
    coord = [row['Lat'],row['Long']]
    marker = folium.Marker(coord, icon=folium.Icon(color='white', icon_color=row['marker_color']))
    marker_cluster.add_child(marker)

site_map

In [12]:
### explore what the launch sites are near (railways, highways, coastline, etc.)

### first, add coordinates to mouse position
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [13]:
### generate a function to triangulate the distance between two sets of lat/long coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [16]:
### calculate the distance between a launch site and its nearest coastline, using coordinates found via Mouse Position
coast_lat, coast_lon = 28.56255, -80.56785
launch_lat, launch_lon = 28.563197, -80.576820 # for CCAFS SLC-40
distance_coastline = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)

### add a text marker for the coastline and show the distance with a line
coord = [coast_lat, coast_lon]
distance_marker = folium.Marker(coord, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), 
                                                    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),))

site_map.add_child(distance_marker)

coords = [[launch_lat,launch_lon],coord]
lines = folium.PolyLine(locations=coords, weight=1)
site_map.add_child(lines)
site_map